# The Neural Bi-gram Model

The logistic regression model that we have considered this far has several considerable disadvantages.

It is slow and expensive to train, despite the fact that it is a linear model. Condider the $V \times V$ weights matrix. It has $V^2$ parameters. For a vocabulary of 100,000 words, this is 10 billion parameters. This is a lot of parameters to estimate, and it is not surprising that it takes a long time to train.
  

We can try to work around this by compressing the inputs into a lower dimensional space, which is exactly what neural networks do. We can think of the neural network as a non-linear function that maps the inputs into a lower dimensional space. The neural network is then a linear model in this lower dimensional space.

![Hidden Layer Network](03-one-hidden-layer.png)

This model will have two sets of weights: $W^{(2)}$ ($V \times D)$ and $W^{(1)}$ ($D \times V)$. Choosing $D$ to be much smaller than $V$ will reduce the number of parameters that we need to estimate. With $D = 100$ and $V = 10000$ we have 1 million parameters, which is a lot less than 10 billion.

Let's define the neural network model:
The hidden layer will have $D = 30$ neurons and a $tanh$ activation.

$$
\begin{align}
\mathbf{H} = tanh(\mathbf{X} \mathbf{W^{(2)}}) \\
\hat{Y} = softmax(\mathbf{H}\mathbf{W^{(1)}})
\end{align}
$$

The loss function is the same as before:

$$
J(\mathbf{W}^h, \mathbf{W}^o) = -\frac{1}{N} \sum_{i=1}^N \sum_{j=1}^V Y_{ij} \log \hat{Y}_{ij}
$$

The weights are updated using gradient descent:

$$
\begin{align}
\mathbf{W}^{h, \text{new}} = \mathbf{W}^{h, \text{old}} - \eta \nabla_{\mathbf{W}^h} J \\
\mathbf{W}^{o, \text{new}} = \mathbf{W}^{o, \text{old}} - \eta \nabla_{\mathbf{W}^o} J \\
\end{align}
$$

The gradients are computed using the chain rule and can be shown to be:

$$
\begin{align}
\nabla_{\mathbf{W}^o} J = \mathbf{H}^T (\hat{Y} - Y) \\
\nabla_{\mathbf{W}^h} J = \mathbf{X}^T (\hat{Y} - Y) \odot (1 - \mathbf{H}^2) \mathbf{W}^{oT}
\end{align}
$$

In the above, the $\odot$ operator is the element-wise product of two matrices.

In [1]:
# As before, we start by producing the training data
import numpy as np

# This is the same softmax function as before
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / exp_x.sum(axis=1, keepdims=True)

def train_neural_bigram(sentences: list, V: int, D: int = 100, learning_rate: float = 0.01, epochs: int = 100):
    """
    Train a neural bigram model with tanh activation function
    
    :param sentences: A list sentences. Each sentence is a list of integers corresponding to the indices of the words in the vocabulary
    :param V: The size of the vocabulary
    :param D: The size of the hidden layer (size of the word vectors)
    :param learning_rate: The learning rate of the gradient descent algorithm
    :param epochs: Number of epochs to train the model
    :return: Hidden and output weights and a list of losses at each iteration
    """
    # initialize weights
    Wh = np.random.randn(V, D) / np.sqrt(V)
    Wo = np.random.randn(D, V) / np.sqrt(D)
    
    # A list to store the loss at each iteration
    losses = []
    
    for epoch in range(epochs):
        # shuffle sentences at each epoch
        np.random.shuffle(sentences)
        
        j = 0 # keep track of iterations
        for sentence in sentences:
            # convert sentence into one-hot encoded inputs and targets
            n = len(sentence)
            inputs = np.zeros((n - 1, V))
            targets = np.zeros((n - 1, V))
            inputs[np.arange(n - 1), sentence[:n-1]] = 1
            targets[np.arange(n - 1), sentence[1:]] = 1
            
            # get output predictions
            hidden = np.tanh(inputs.dot(Wh))
            p = softmax(hidden.dot(Wo))
            
            # do a gradient descent step
            W2 = Wo - learning_rate * hidden.T.dot(p - targets)
            dhidden = (p - targets).dot(W2.T) * (1 - hidden * hidden)
            Wh = Wh - learning_rate * inputs.T.dot(dhidden)
            
            # keep track of the loss
            loss = -np.sum(targets * np.log(p)) / (n - 1)
            losses.append(loss)
            
            if j % 10 == 0:
                print("epoch:", epoch, "sentence: %s/%s" % (j, len(sentences)), "loss:", loss)
            j += 1
            
    return Wh, Wo, losses

# The Skip-Gram Model

In the previous model, we used the first word in a bi-gram to predict the second word. 

$$
P(w_{t+1} | w_t) = \text{softmax}(\mathbf{W}^o^T f(\mathbf{W}^hT x_t))
$$

The model already produces word vectors (embeddings) for each word in the vocabulary. A problem with the model, however, is that these
word vectors perform poorly on word similarity and other NLP tasks, because they are trained to predict the next word in a sequence, nothing else.

The skip-gram model is a modification of the bi-gram model that produces better word vectors. Instead of predicting the next word, it uses the current word to predict its surrounding words (context window).

Furthermore, it uses the dot product between the sets of weights (word vectors) to compute the probability of the surrounding words. The dot-product is related to the similarity (cosine) of the word vectors.


Let's look at an example sentence:

```
Alice was beginning to get very **tired** of sitting by her sister on the bank.
```

The bi-gram model would produce the following training data the word "tired":

- (tired, of)

The skip-gram model looks at a set of nearby words (context window). For a context window of size $w = 2$, the skip-gram will produce the following training data for the word tired:

- (tired, get)
- (tired, very)
- (tired, of)
- (tired, sitting)

You can think about the model in two ways

1. One sample with four targets
    - tired -> (very, get, of, sitting)
2. Four samples with a single target each
    - tired -> very
    - tired -> get
    - tired -> of
    - tired -> sitting


Let's define two matrices of weights $W^{(1)} \in M(V, D)$ that maps a one-hot encoded (sparse) representation of a word to a $D$-dimensional (dense) representation of that word. Let $W^{(2)} \in M(D, V)$ be a matrix of weights that maps a $D$-dimensional representation of a word to a $V$ dimensional output vector.

For a vocabulary of size $V = 3$ and embedding size $D = 2$, the two matrices of weights will be:

$$
W^{(1)} = \begin{pmatrix}
    w^{(1)}_{11} & w^{(1)}_{12}  \\
    w^{(1)}_{21} & w^{(1)}_{22} \\
    w^{(1)}_{31} & w^{(1)}_{32}
\end{pmatrix}
$$

The matrix of weights for the output layer will be:

$$
W^{(2)} = \begin{pmatrix}
    w^{(2)}_{11} & w^{(2)}_{12} & w^{(2)}_{13} \\
    w^{(2)}_{21} & w^{(2)}_{22} & w^{(2)}_{23} \\
\end{pmatrix}
$$

Let's look at a single training sample (center word, context word pair) for the word "tired":
and let the one-hot encoded representation of the word "tired" be:
$$
(0, 1, 0)
$$

The hidden layer will be:

$$
H = (0, 1, 0) \begin{pmatrix}
    w^{(1)}_{11} & w^{(1)}_{12}  \\
    w^{(1)}_{21} & w^{(1)}_{22} \\
    w^{(1)}_{31} & w^{(1)}_{32}
\end{pmatrix} = (w^{(1)}_{21}, w^{(1)}_{22})
$$

Notice that because of the on-hot encoded nature of the input word vector, the hidden layer is just the second row of the matrix $W^{(1)}$. We can use this fact in the software implementation of the model to avoid the matrix multiplication (and save compute time).


The output layer will be:

$$
\begin{align}
a & = (w^{(1)}_{21}, w^{(1)}_{22}) \begin{pmatrix}
    w^{(2)}_{11} & w^{(2)}_{12} & w^{(2)}_{13} \\
    w^{(2)}_{21} & w^{(2)}_{22} & w^{(2)}_{23} \\
\end{pmatrix} \\
& = \begin{pmatrix}
  w^{(1)}_{21} w^{(2)}_{11} + w^{(1)}_{22} w^{(2)}_{21} \\ 
  w^{(1)}_{21} w^{(2)}_{12} + w^{(1)}_{22} w^{(2)}_{22} \\
  w^{(1)}_{21} w^{(2)}_{13} + w^{(1)}_{22} w^{(2)}_{23}) \\
  \end{pmatrix}^{T}
\end{align}
$$
The transpose in the last step is there only for the sake of readability. The output layer is a vector of size $V = 3$ that contains the probabilities of the context words given the center word. The softmax function is applied to the output layer to produce the probabilities:

$$
\text{softmax}(a) = \begin{pmatrix}
  \frac{e^{a_1}}{e^{a_1} + e^{a_2} + e^{a_3}} \\
  \frac{e^{a_2}}{e^{a_1} + e^{a_2} + e^{a_3}} \\
  \frac{e^{a_3}}{e^{a_1} + e^{a_2} + e^{a_3}} \\
  \end{pmatrix} =
  \begin{pmatrix}
  p_1 \\
  p_2 \\
  p_3 \\
  \end{pmatrix}
$$

These probabilities are the predicted probabilities of the context words given the center word. These probabilities can be compared to the one-hot encoded representation of the context words to compute the loss function and we can apply gradient descent to update the weights.

Although theoretically sound, this approach suffers from a couple of practical difficulties.

The softmax function is computationally expensive. The softmax function is applied to a vector of size $V$ and involves exponentiation of the elements of the vector. For a large vocabulary size $V$, e.g. a coupe of millions this can be computationally expensive. The pretrained word2vec vectors provided by google have a vocabulary size of about 4 million! This means that a lot oc compute time will be spent on working out the softmax function.

The authors of the word2vec models proposed a couple of solutions to this problem. The first solution is to use a hierarchical softmax function instead of the softmax function. The hierarchical softmax function is a binary tree of words where each node is a binary classifier that predicts whether the word is in the left or right branch of the tree. 

Here we will focus on the second approach that is called negative sampling. 


## Negative Sampling


Let's look at the predictions that skip-gram model makes for a context word. For any given center word, most words in the vocabulary are not context words. For example, for the center word "tired", the words "very", "get", "of" and "sitting" are context words, but the words "apple", "car", "house" and "computer" are not context words. The skip-gram model will predict a high probability for the context words and a low probability for the non-context words.

In order to reduce the computational complexity of the softmax function, the authors of the word2vec models proposed to train the model to predict the context words and a small number of random words that are not in the context in a binary classification problem.

Instead of doing full multi-class classification with softmax, we can train the model to do binary classification. For each training sample (center word, context word pair), we will create a new training sample where the center word is paired with a random word from the vocabulary that is not in the context. The new training sample will have a label of 1 if the random word is in the context and a label of 0 if the random word is not in the context. We will train the model to predict the label of the new training sample.


Let's see an example. Consider the training sample (center word, context word pair) for the word "tired" that we saw before. In the following, $\sigma$ stands for the sigmoid function (which we use to produce probabilities).

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

In our example the logistic regressions for the center word "tired" will be:

- (tired, very): $p(very | tired) = \sigma(W^{(1)}_{\text{very}}W^{o}_{\text{tired}})$
- (tired, get): $p(get | tired) = \sigma(W^{(1)}_{\text{get}}W^{o}_{\text{tired}})$
- (tired, very): $p(of | tired) = \sigma(W^{(1)}_{\text{of}}W^{o}_{\text{tired}})$
- (tired, very): $p(sitting | tired) = \sigma(W^{(1)}_{\text{sitting}}W^{o}_{\text{tired}})$

By sampling from the modified uni-gram distribution we can select a couple (hyperparameter) of negative examples for each center word. For example, we can sample the words "cat", "dog", "mouse", "rabbit", and "horse" as negative examples for the center word "tired". The logistic regressions for the negative examples will be:

- (tired, cat): $p(cat | tired) = \sigma(W^{(1)}_{\text{cat}}W^{o}_{\text{tired}})$
- (tired, dog): $p(dog | tired) = \sigma(W^{(1)}_{\text{dog}}W^{o}_{\text{tired}})$
- (tired, mouse): $p(mouse | tired) = \sigma(W^{(1)}_{\text{mouse}}W^{o}_{\text{tired}})$
- (tired, rabbit): $p(rabbit | tired) = \sigma(W^{(1)}_{\text{rabbit}}W^{o}_{\text{tired}})$
- (tired, horse): $p(horse | tired) = \sigma(W^{(1)}_{\text{horse}}W^{o}_{\text{tired}})$


The loss function with negative sampling is:

$$
J = \log p(very | tired) + \log p(get | tired) + \log p(of | tired) + \log p(sitting | tired) + \log (1 - p(cat | tired)) + \log (1 - p(dog | tired)) + \log (1 - p(mouse | tired)) + \log (1 - p(rabbit | tired)) + \log (1 - p(horse | tired))
$$

Generalizing the loss will give us

$$
J = \sum_{j \in \text{pos. examples}} \log \sigma(W^{(1)}_jW^{(2)}) + \sum_{j \in \text{neg. examples}} \log (1 - \sigma(W^{(1)}_jW^{(2)}))
$$

We can use a property of the sigmoid function to simplify the loss function:

$$
\sigma(x) + \sigma(-x) = 1
$$

$$
J = \sum_{j \in \text{pos. examples}} \log \sigma(W^{(1)}_jW^{(2)}) + \sum_{j \in \text{neg. examples}} \log (\sigma(-W^{(1)}_jW^{(2)}))
$$


## Unigram Distribution

We introduced the negative sampling technique to reduce the computational cost of the softmax function but the question remains how to choose the negative examples. The authors of the skip-gram model suggest using the uni-gram distribution raised to the 3/4 power.

The uni-gram distribution is simply the frequency of each word in the corpus. Let $f(v)$ be the frequency of the $v$-th word in the corpus. The probability of sampling the $v$-th word is:

$$
\text{f}_{3/4}(v) = f(v)^{3/4} \\ 
p_{\text{neg}}(v) = \frac{f_{3/4}}{\sum_{v' = 1}^{V} f_{3/4}(v')}
$$

The basic idea of raising the frequencies to the 3/4 power is to reduce the probability of sampling frequent words. The authors of the skip-gram model found that this distribution works well empirically, but there is no theoretical justification for it. It is a hyperparameter of the model, and it can be tuned using a validation set (e.g. checking how the word-vectors perform in word analogy tests).


## Gradient Descent with Negative Sampling

For a set of positive and negative examples, the $t_n$ equal to 1 for positive examples and 0 for negative examples. Furthermore, let $p_n$ be the predicted probability that a word is in the context of the center word.

$$
p(y = 1 | center) = \sigma(W^{(2)T}_{target}W^{(1)}_{center}) \\
$$
The loss function for a single training sample is therefore:

$$
J = - t_{n} \log (y_n = 1 | center) - (1 - t_{n}) \log (1 - p(y_n = 1 | center))
$$

For a batch of $N$ training samples, the loss function is:

$$
J = - \sum_{n = 1}^{N} t_{n} \log p_n + (1 - t_{n}) \log (1 - p_n)
$$

In order to understand the weight update process it is easier to consider the loss function for a single training sample: a center word and a target word.

$$
J_n = - t_{n} \log p_n + (1 - t_{n}) \log (1 - p_n)
$$


In order to use gradient descent to minimize the loss function, we need to compute the gradient of the loss function with respect to the weights. In the following $W^{(2)}_n$ is a $D$-dimensional vector that is the $n$-th column of the weight matrix $W^{(2)}$.

$$
\begin{align}
\frac{\partial}{\partial W^{(2)}_n} J & = \frac{\partial}{\partial W^{(2)}_n} \left( -  t_{n} \log p_n - (1 - t_{n}) \log (1 - p_n) \right) \\
& = - \left( \frac{t_{n}}{p_n} \frac{\partial p_n}{\partial W^{(2)}_n} - \frac{1 - t_{n}}{1 - p_n} \frac{\partial p_n}{\partial W^{(2)}_n} \right)\\
& = - \left( \frac{t_{n}}{p_n} - \frac{1 - t_{n}}{1 - p_n} \right) \frac{\partial p_n}{\partial W^{(2)}_n}
\end{align}
$$

The probability $p_n$ is just the sigmoid function applied to the net value of the output layer:

$$
p_n = \sigma(a_n)
$$
We can apply the chain rule to compute the derivative of the loss function with respect to the weights:


$$
\frac{\partial p_n}{\partial a_n} = \frac{\partial \sigma(a_n)}{\partial a_n} = \sigma(a_n)(1 - \sigma(a_n))
$$

$$
\begin{align}
\frac{\partial}{\partial W^{(2)}_n} J_n & = - \left( \frac{t_{n}}{p_n} - \frac{1 - t_{n}}{1 - p_n} \right) \frac{\partial p_n}{\partial W^{(2)}_n} \\
& = - \left( \frac{t_{n}}{p_n} - \frac{1 - t_{n}}{1 - p_n} \right) \frac{\partial p_n}{\partial a_n} \frac{\partial a_n}{\partial W^{(2)}_n} \\
& = - \left( \frac{t_{n}}{p_n} - \frac{1 - t_{n}}{1 - p_n} \right) \sigma(a_n)(1 - \sigma(a_n)) \frac{\partial a_n}{\partial W^{(2)}_n} \\
& = - \left( \frac{t_{n}}{p_n} - \frac{1 - t_{n}}{1 - p_n} \right) p_n(1 - p_n) \frac{\partial a_n}{\partial W^{(2)}_n} \\
& = - \left( t_n (1 - p_n) - (1 - t_n)p_n \right) \frac{\partial a_n}{\partial W^{(2)}_n} \\
& = - \left( t_n - t_np_n - p_n + t_np_n \right) \frac{\partial a_n}{\partial W^{(2)}_n} \\
& = - \left( t_n - p_n \right) \frac{\partial a_n}{\partial W^{(2)}_n} \\
\end{align}
$$

$$
a_n = W^{(2)T}_{n}W^{(1)}_{center}
$$
The derivative of the net value of the output layer with respect to the weights is:

$$
\frac{\partial a_n}{\partial W^{(2)}_n} = W^{(1)}_{center}
$$
In the end, the gradient of the loss function with respect to the output layer weights is:

$$
\begin{align}
\frac{\partial}{\partial W^{(2)}_n} J & = \left( t_n - p_n \right) W^{(1)}_{\text{center}} \\
\end{align}
$$

In our software implementation we will want to compute the gradient of the loss function for a whole batch of training samples (positive samples and negative samples), so it is 
convenient (and much faster computationally) to express the gradient for the batch in matrix form.

For a batch of $N = 3$ training samples, the gradients could be the following (for example):

$$
\begin{align}
\frac{J}{\partial W^{(2)}_{of}} & = W^{(1)}_{tired} (p_{of} - t_{of}) \\
\frac{J}{\partial W^{(2)}_{sitting}} & = W^{(1)}_{tired} (p_{sitting} - t_{sitting}) \\
\frac{J}{\partial W^{(2)}_{car}} & = W^{(1)}_{tired} (p_{car} - t_{car}) \\
\end{align}
$$

This time we will need to update three ($N$) columns of the weight matrix $W^{(2)}$ that
correspond to the target words. We can express the gradient of the loss function
with respect to the $N \times D$ weights that need to be updated as the outer
product of the $D$-dimensional vector $W^{(1)}_{tired}$ and the $N$-dimensional
vector of errors $P_C - T_C$. With this notation $C$ stands for the set of target
words. $\otimes$ denotes the outer product.

$$
\frac{\partial J}{\partial W^{(2)}_C} = W^{(1)}_{tired} \otimes (P_{C} - T_{C})
$$

To recall, the outer product of an $m$-dimensional vector $a$ and an $n$-dimensional vector $b$ is an $m \times n$ matrix with the following form:

$$
\begin{pmatrix}
a_1 \\
a_2 \\
\vdots \\
a_m
\end{pmatrix}
\otimes 
\begin{pmatrix}
b_1 \\
b_2 \\
\vdots \\
b_n
\end{pmatrix}
= \begin{pmatrix}
a_1 b_1 & a_1 b_2 & \cdots & a_1 b_n \\
a_2 b_1 & a_2 b_2 & \cdots & a_2 b_n \\
\vdots & \vdots & \ddots & \vdots \\
a_m b_1 & a_m b_2 & \cdots & a_m b_n \\
\end{pmatrix}
$$

Next we need to find the gradient of the loss function with respect to the center word vector.
As before, we will derive it for a single training sample, and then we will generalize it for a batch of training samples. The good news is that we can reuse the result from the previous section.


$$
\begin{align}
\frac{\partial J}{\partial a_n} & = p_n - t_n \\
\end{align}
$$
Now we need to see that the loss function $J$ depends on the center word vector $W^{(1)}_{center}$ through net output values of all samples in the batch $a_n$. Therefore, the derivative of the loss function with respect to the center word vector is:

$$
\begin{align}
\frac{\partial J}{\partial W^{(1)}_{center}} & = \sum_{n=1}^{N} \frac{\partial J}{\partial a_n} \frac{\partial a_n}{\partial W^{(1)}_{center}} \\
& = \sum_{n=1}^{N} (p_n - t_n) \frac{\partial}{\partial W^{(1)}_{center}}\left(
W^{(2)T}_{n}W^{(1)}_{center}
\right) \\
& = \sum_{n=1}^{N} (p_n - t_n) W^{(2)}_{n} \\
\end{align}
$$

## Subsampling Frequent Words

There is one final detail of the skip-gram model that we need to discuss.

A pattern occurring in most human language texts is that some words are much more frequent than others. There is an empirical finding known as Zipf's law that states that the frequency of a word is inversely proportional to its rank in the frequency table. For example, the most frequent word will occur twice as often as the second most frequent word, three times as often as the third most frequent word, and so on.

$$
\text{frequency} \propto \frac{1}{\text{rank}}
$$

In the English language, "the" is by far the most common word. It accounts for nearly 7% of all words in a typical text. The second most common word "of" accounts for slightly over 3% of words, followed by "and" which accounts for slightly over 2% of words.

In the context of the skip-gram model this means that the model will spend a lot of time updating the weights for the word "the" and very little time updating the weights for the word "temporary" (for example). We would like to spend more time updating the weights for rare words and less time updating the weights for frequent words, especially because the most 
common words are often function words that do not carry much meaning.

A way to work around this problem is to use only a part of the words in a sentence for training in each epoch. We would certainly not like to exclude the same words every time, because this would break the context of the center word. So we will take a random sample of words from the sentence. Thus, for each epoch the sample of words will be different. This is called subsampling.

The subsampling distribution proposed in word2vec is the following:

$$
P(v) = 1 - \sqrt{\frac{t}{f(v)}}
$$

where $t$ is a threshold parameter and $f(v)$ is the frequency of the word $v$ in the training corpus. The threshold parameter $t$ is usually set to $10^{-5}$. Again, this is a hyperparameter that can be tuned.
